# Setup/Test Environment:

In [1]:
from web_footprinting_utils import setup_ipython
import os

get_ipython, INTERACTIVE = setup_ipython()

In [2]:
%env TARGET=10.10.165.36

env: TARGET=10.10.165.36


In [3]:
!echo Make sure we are connected to VPN:
result = !ip a | grep tun0

if not len(result)>0:
    print("Not connected!")
    input()
else:
    print("Connected!")
    print(result)

Make sure we are connected to VPN:
Not connected!


# Initial (Unreliable) RustScan:

In [4]:
import os
pwd = os.getcwd()
rustscan = "docker run -v '%s':/workspace  --volume /etc/passwd:/etc/passwd:ro --volume /etc/group:/etc/group:ro --user $(id -u) --rm my-other-github-account/rustscan" % pwd

In [5]:
!{rustscan} -t 4000 -r 1-65535 -a {os.environ["TARGET"]} -- -Pn -A -sV

/bin/bash: docker: command not found


# Initial Nmap:

In [3]:
!echo $TARGET
!mkdir -p scans/quick scans/tcp scans/udp scans/scripts scans/http scans/no_ping

10.10.165.36


In [7]:
!ping -c 5 $TARGET

PING 10.10.207.10 (10.10.207.10) 56(84) bytes of data.
64 bytes from 10.10.207.10: icmp_seq=1 ttl=128 time=0.443 ms
64 bytes from 10.10.207.10: icmp_seq=2 ttl=128 time=0.431 ms
64 bytes from 10.10.207.10: icmp_seq=3 ttl=128 time=1.39 ms
64 bytes from 10.10.207.10: icmp_seq=4 ttl=128 time=0.436 ms
64 bytes from 10.10.207.10: icmp_seq=5 ttl=128 time=0.487 ms

--- 10.10.207.10 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4069ms
rtt min/avg/max/mdev = 0.431/0.638/1.394/0.378 ms


# Quick Scan:

In [9]:
!nmap -vv -oA scans/quick/quick_scan $TARGET

Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-04 01:21 UTC
Initiating ARP Ping Scan at 01:21
Scanning 10.10.165.36 [1 port]
Completed ARP Ping Scan at 01:21, 0.05s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 01:21
Completed Parallel DNS resolution of 1 host. at 01:21, 0.00s elapsed
Initiating SYN Stealth Scan at 01:21
Scanning ip-10-10-165-36.eu-west-1.compute.internal (10.10.165.36) [1000 ports]
Discovered open port 10000/tcp on 10.10.165.36
Discovered open port 9999/tcp on 10.10.165.36
Completed SYN Stealth Scan at 01:21, 0.12s elapsed (1000 total ports)
Nmap scan report for ip-10-10-165-36.eu-west-1.compute.internal (10.10.165.36)
Host is up, received arp-response (0.0043s latency).
Scanned at 2021-10-04 01:21:25 UTC for 0s
Not shown: 998 closed ports
Reason: 998 resets
PORT      STATE SERVICE          REASON
9999/tcp  open  abyss            syn-ack ttl 64
10000/tcp open  snet-sensor-mgmt syn-ack ttl 64
MAC Address: 02:EB:CF:EF:F4:31 (Unknown)

Read

# All Ports:

In [9]:
!nmap -p- -sV -vv -oA  scans/tcp/tcp_scan $TARGET

Starting Nmap 7.80 ( https://nmap.org ) at 2021-09-30 00:46 UTC
NSE: Loaded 45 scripts for scanning.
Initiating ARP Ping Scan at 00:46
Scanning 10.10.207.10 [1 port]
Completed ARP Ping Scan at 00:46, 0.03s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 00:46
Completed Parallel DNS resolution of 1 host. at 00:46, 0.00s elapsed
Initiating SYN Stealth Scan at 00:46
Scanning ip-10-10-207-10.eu-west-1.compute.internal (10.10.207.10) [65535 ports]
Discovered open port 139/tcp on 10.10.207.10
Discovered open port 135/tcp on 10.10.207.10
Discovered open port 445/tcp on 10.10.207.10
Increasing send delay for 10.10.207.10 from 0 to 5 due to 24 out of 78 dropped probes since last increase.
SYN Stealth Scan Timing: About 8.87% done; ETC: 00:52 (0:05:18 remaining)
SYN Stealth Scan Timing: About 17.78% done; ETC: 00:52 (0:04:42 remaining)
SYN Stealth Scan Timing: About 26.68% done; ETC: 00:52 (0:04:10 remaining)
Discovered open port 49162/tcp on 10.10.207.10
Discovered open

In [10]:
import xmltodict

with open('scans/tcp/tcp_scan.xml','r') as f:
    xml_dict = xmltodict.parse(f.read())

In [11]:
PING=True
try:
    port_list = [port['@portid'] for port in xml_dict['nmaprun']['host']['ports']['port']]
    %env PORTS={",".join(port_list)}
except:
    print("Host is down")
    PING=False

env: PORTS=135,139,445,31337,49152,49153,49154,49160,49161,49162


# No Ping:

In [12]:
!nmap -sV -Pn -vv -oA scans/no_ping/no_ping_scan $TARGET

Starting Nmap 7.80 ( https://nmap.org ) at 2021-09-30 00:54 UTC
NSE: Loaded 45 scripts for scanning.
Initiating ARP Ping Scan at 00:54
Scanning 10.10.207.10 [1 port]
Completed ARP Ping Scan at 00:54, 0.04s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 00:54
Completed Parallel DNS resolution of 1 host. at 00:54, 0.00s elapsed
Initiating SYN Stealth Scan at 00:54
Scanning ip-10-10-207-10.eu-west-1.compute.internal (10.10.207.10) [1000 ports]
Discovered open port 139/tcp on 10.10.207.10
Discovered open port 445/tcp on 10.10.207.10
Discovered open port 3389/tcp on 10.10.207.10
Discovered open port 135/tcp on 10.10.207.10
Increasing send delay for 10.10.207.10 from 0 to 5 due to 45 out of 149 dropped probes since last increase.
Discovered open port 49160/tcp on 10.10.207.10
Discovered open port 31337/tcp on 10.10.207.10
Discovered open port 49154/tcp on 10.10.207.10
Discovered open port 49161/tcp on 10.10.207.10
Discovered open port 49153/tcp on 10.10.207.10
Disco

In [13]:
if not PING:
    !nmap -sV -Pn -p- -vv -oA  scans/tcp/tcp_scan $TARGET

# Run Default Scripts:

In [14]:
if PING:
    !nmap -sC -p $PORTS -vv -oA scans/scripts/script_scan $TARGET

Starting Nmap 7.80 ( https://nmap.org ) at 2021-09-30 00:57 UTC
NSE: Loaded 121 scripts for scanning.
NSE: Script Pre-scanning.
NSE: Starting runlevel 1 (of 2) scan.
Initiating NSE at 00:57
Completed NSE at 00:57, 0.00s elapsed
NSE: Starting runlevel 2 (of 2) scan.
Initiating NSE at 00:57
Completed NSE at 00:57, 0.00s elapsed
Initiating ARP Ping Scan at 00:57
Scanning 10.10.207.10 [1 port]
Completed ARP Ping Scan at 00:57, 0.03s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 00:57
Completed Parallel DNS resolution of 1 host. at 00:57, 0.00s elapsed
Initiating SYN Stealth Scan at 00:57
Scanning ip-10-10-207-10.eu-west-1.compute.internal (10.10.207.10) [10 ports]
Discovered open port 445/tcp on 10.10.207.10
Discovered open port 139/tcp on 10.10.207.10
Discovered open port 49160/tcp on 10.10.207.10
Discovered open port 135/tcp on 10.10.207.10
Discovered open port 31337/tcp on 10.10.207.10
Discovered open port 49152/tcp on 10.10.207.10
Discovered open port 49154/t

In [15]:
if PING:
    !nmap -A -T5 -vv --script=default,safe,discovery $TARGET
else:
    !nmap -A -T5 -Pn -vv --script=default,safe,discovery $TARGET

Starting Nmap 7.80 ( https://nmap.org ) at 2021-09-30 00:59 UTC
NSE: Loaded 417 scripts for scanning.
NSE: Script Pre-scanning.
NSE: Starting runlevel 1 (of 3) scan.
Initiating NSE at 00:59
NSE: [shodan-api] Error: Please specify your ShodanAPI key with the shodan-api.apikey argument
NSE: [targets-ipv6-map4to6] This script is IPv6 only.
NSE: [broadcast-ataoe-discover] No interface supplied, use -e
NSE: [mtrace] A source IP must be provided through fromip argument.
NSE: [broadcast-sonicwall-discover] No network interface was supplied, aborting.
NSE: [targets-xml] Need to supply a file name with the targets-xml.iX argument
NSE: [targets-ipv6-wordlist] Need to be executed for IPv6.
NSE: [url-snarf] no network interface was supplied, aborting ...
too short
NSE Timing: About 98.85% done; ETC: 00:59 (0:00:00 remaining)
Completed NSE at 01:00, 40.04s elapsed
NSE: Starting runlevel 2 (of 3) scan.
Initiating NSE at 01:00
Completed NSE at 01:00, 0.00s elapsed
NSE: Starting runlevel 3 (of 3) scan

nmap: nse_nsock.cc:369: void callback(nsock_pool, nsock_event, void*): Assertion `lua_status(L) == LUA_YIELD' failed.


In [16]:
if PING:
    !nmap -A -T3 -vv --script=default,safe,discovery $TARGET
else:
    !nmap -A -T3 -Pn -vv --script=default,safe,discovery $TARGET

Starting Nmap 7.80 ( https://nmap.org ) at 2021-09-30 01:03 UTC
NSE: Loaded 417 scripts for scanning.
NSE: Script Pre-scanning.
NSE: Starting runlevel 1 (of 3) scan.
Initiating NSE at 01:03
NSE: [broadcast-sonicwall-discover] No network interface was supplied, aborting.
NSE: [targets-ipv6-wordlist] Need to be executed for IPv6.
NSE: [url-snarf] no network interface was supplied, aborting ...
NSE: [targets-xml] Need to supply a file name with the targets-xml.iX argument
NSE: [mtrace] A source IP must be provided through fromip argument.
NSE: [broadcast-ataoe-discover] No interface supplied, use -e
NSE: [targets-ipv6-map4to6] This script is IPv6 only.
NSE: [shodan-api] Error: Please specify your ShodanAPI key with the shodan-api.apikey argument
too short
NSE Timing: About 98.85% done; ETC: 01:04 (0:00:00 remaining)
Completed NSE at 01:04, 40.03s elapsed
NSE: Starting runlevel 2 (of 3) scan.
Initiating NSE at 01:04
Completed NSE at 01:04, 0.00s elapsed
NSE: Starting runlevel 3 (of 3) scan

nmap: nse_nsock.cc:369: void callback(nsock_pool, nsock_event, void*): Assertion `lua_status(L) == LUA_YIELD' failed.


In [ ]:
from getpass import getpass
import os

if PING:
    !echo { getpass() } | sudo -S nmap -A -p- -T5 -vv --script=default,safe,discovery {os.environ['TARGET']}
else:
    !echo { getpass() } | sudo -S nmap -A -p- -Pn -T5 -vv --script=default,safe,discovery {os.environ['TARGET']}

In [ ]:
# UDP Scan, this takes forever...

# from getpass import getpass

# !echo { getpass() } > tmp.txt
# !cat tmp.txt | sudo -S nmap -sU --open -oA scans/udp/udp_scan $TARGET
# !cat tmp.txt | sudo -S nmap -sU --open -p- -oA scans/udp/udp_scan $TARGET
# !rm -rf tmp.txt